In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import ipyparallel as ipp
print(ipp.version_info)

nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

(8, 6, 1)
<Cluster(cluster_id='1690369408-my6r', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.50s/engine]


[0, 1, 2, 3]

In [3]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1690369408-my6r                  True          4 MPI


In [4]:
%autopx --block --group-outputs=engines

%autopx enabled


In [5]:
import numpy as np
from qtm.constants import RYDBERG

from qtm.lattice import RealLattice, ReciLattice
from qtm.gspace import GSpace, GkSpace

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

ecutwfn = 40 * RYDBERG
grho = GSpace(recilat, 4 * ecutwfn)
gwfn = GkSpace(grho, [0., 0., 0.])

In [6]:
arr_g = grho.create_buffer_g(1)
arr_r = grho.create_buffer_r(1)

rng = np.random.default_rng(12345)
arr_g[:] = rng.random(grho.size_g) \
    + 1j * rng.random(grho.size_g)

arr_r[:] = 0
arr_r.put(grho.idxgrid, arr_g)

arr_r = np.fft.ifftn(arr_r.reshape(grho.grid_shape))
arr_r = arr_r.reshape((1, -1))

out = grho.create_buffer_r(1)
grho.g2r(arr_g, out)
print(np.allclose(out, arr_r))

out = grho.create_buffer_g(1)
grho.r2g(arr_r, out)
print(np.allclose(out, arr_g))

[stdout:2] True
True


[stdout:1] True
True


[stdout:3] True
True


[stdout:0] True
True


In [7]:
from mpi4py.MPI import COMM_WORLD
from qtm.mpi.comm import QTMComm, split_comm_pwgrp

pwgrp_size = 4

world_comm = QTMComm(COMM_WORLD)
pwgrp_comm, intercomm = split_comm_pwgrp(world_comm, pwgrp_size)
n_pwgrp = intercomm.size

print(f"{world_comm.rank}/{world_comm.size}: "
      f"PWGRP #{pwgrp_comm.subgrp_idx}/{n_pwgrp} "
      f"pwgrpcomm rank {pwgrp_comm.rank}/{pwgrp_size} - "
      f"intercomm rank {intercomm.rank}/{intercomm.size}")

[stdout:3] 3/4: PWGRP #0/1 pwgrpcomm rank 3/4 - intercomm rank 0/1


[stdout:0] 0/4: PWGRP #0/1 pwgrpcomm rank 0/4 - intercomm rank 0/1


[stdout:1] 1/4: PWGRP #0/1 pwgrpcomm rank 1/4 - intercomm rank 0/1


[stdout:2] 2/4: PWGRP #0/1 pwgrpcomm rank 2/4 - intercomm rank 0/1


In [8]:
from qtm.mpi.gspace import DistGSpace

dist_grho = DistGSpace(pwgrp_comm, grho)
print(dist_grho.ecut)

[stdout:2] 80.0


[stdout:1] 80.0


[stdout:0] 80.0


[stdout:3] 80.0


In [11]:
from qtm.mpi.utils import scatter_slice
out = dist_grho.scatter_data_r(arr_r)

sl = scatter_slice(dist_grho.nz, pwgrp_comm.size, pwgrp_comm.rank)
arr_r_dist = arr_r.reshape(grho.grid_shape)[..., sl].ravel()
print(np.allclose(arr_r_dist, out))

[stdout:1] True


[stdout:0] True


[stdout:3] True


[stdout:2] True


In [10]:
# print(dist_grho._scatter_r_sendcounts)

In [12]:
%autopx --block --group-outputs=engines

%autopx disabled


In [13]:
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 26557, 'identifier': 'ipcontroller-1690369408-my6r-26542'}
Stopping engine(s): 1690369409
engine set stopped 1690369409: {'exit_code': 1, 'pid': 26586, 'identifier': 'ipengine-1690369408-my6r-1690369409-26542'}


In [15]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1690370528-elmy                  True          0 MPI
